<a href="https://colab.research.google.com/github/plaban1981/Agents/blob/main/Audio_powered_RAG_using_langchain_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required libraries

In [1]:
!pip install langchain langchain_community langchain_groq chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [1]:
!pip install unstructured
!pip install "unstructured[pdf]"
!pip install -U langchain-huggingface

In [34]:
!pip install -U langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: chroma-hnswlib
    Found existing installation: chroma-hnswlib 0.7.6
    Uninstalling chroma-hnswlib-0.7.6:
      Successfully uninstalled chroma-hnswlib-0.7.6
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.5
    Uninstalling chromadb-0.5.5:
      Successfully uninstalled chromadb-0.5.5


In [10]:
!pip install -U langchain-huggingface

## Setup Groq API Key

In [1]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [5]:
!sudo apt-get install python3-pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2
Suggested packages:
  python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 python3-pyaudio
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 91.2 kB of archives.
After this operation, 340 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pyaudio amd64 0.2.11-1.3ubuntu1 [25.9 kB]
Fetched 91.2 kB in 0s (192 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline

In [5]:
from google.colab import output
from IPython.display import Javascript
from base64 import b64decode

# JavaScript code to record audio
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))

const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async () => {
        blob = new Blob(chunks)
        text = await b2text(blob)
        resolve(text)
    }
    recorder.stop()
})
"""

def record(sec=3):
    print("Recording...")
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec * 1000))
    print("Done recording!")
    b = b64decode(s.split(',')[1])
    return b

# Record audio for 5 seconds
audio_data = record(5)

# Save the audio data to a .wav file
with open('recorded_audio.wav', 'wb') as f:
    f.write(audio_data)

print("Audio saved as 'recorded_audio.wav'.")

Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.


## Transcribe Audio using Groq distil-whisper-large-v3-en

#### distil-whisper-large-v3-en :
Distil-Whisper English is a distilled, or compressed, version of OpenAI's Whisper model, designed to provide faster, lower cost English speech recognition while maintaining comparable accuracy.

#### Supported Language : English only

In [6]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq()

# Specify the path to the audio file
filename = "/content/recorded_audio.wav"

def transcribe_audio(filename):
  # Open the audio file
  with open(filename, "rb") as file:
      # Create a transcription of the audio file
      transcription = client.audio.transcriptions.create(
        file=(filename, file.read()), # Required audio file
        model="distil-whisper-large-v3-en", # Required model to use for transcription
        prompt="Specify context or spelling",  # Optional
        response_format="json",  # Optional
        language="en",  # Optional
        temperature=0.0  # Optional
      )
      # Print the transcription text
      print(transcription.text)
  return transcription.text



In [7]:
transcription = transcribe_audio(filename)

 Fibromyalgia.


## Set up the LLM

In [8]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama-3.1-70b-versatile",
    temperature=0.1,
    max_tokens=1000,
)

## Set up the Embedding Model

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
model_name ="BAAI/bge-small-en-v1.5"
model_kwargs ={"device":"cpu"}
encode_kwargs ={"normalize_embeddings":False}
embeddings = HuggingFaceEmbeddings(model_name=model_name,
                                   model_kwargs=model_kwargs,
                                   encode_kwargs=encode_kwargs)

## Download Data

In [23]:
mkdir "Data"

In [26]:
!wget "https://www.torbayandsouthdevon.nhs.uk/uploads/what-is-fibromyalgia.pdf" -O "Data/input.pdf"

--2024-08-25 06:54:42--  https://www.torbayandsouthdevon.nhs.uk/uploads/what-is-fibromyalgia.pdf
Resolving www.torbayandsouthdevon.nhs.uk (www.torbayandsouthdevon.nhs.uk)... 91.197.230.43
Connecting to www.torbayandsouthdevon.nhs.uk (www.torbayandsouthdevon.nhs.uk)|91.197.230.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254897 (249K) [application/pdf]
Saving to: ‘Data/input.pdf’

Data/input.pdf      100%[===================>] 248.92K   805KB/s    in 0.3s    

2024-08-25 06:54:43 (805 KB/s) - ‘Data/input.pdf’ saved [254897/254897]



## Set up Textsplitter

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def text_splitter():
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=512,
      chunk_overlap=20,
      length_function=len,
  )
  return text_splitter

## Read the Input data and split the data into chunks

In [13]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("Data/input.pdf")
pages = loader.load_and_split(text_splitter())

In [14]:
len(pages)

46

In [15]:
pages[0].metadata

{'source': 'Data/input.pdf', 'page': 0}

## Setup the Vector Store

In [16]:
from langchain_chroma import Chroma
vectorstore = Chroma(embedding_function=embeddings,
                     persist_directory="./chromadb",
                     collection_name="fibromylagia")

## add documents

In [17]:
vectorstore.add_documents(pages)

['3406f0a4-6ad3-405a-bd13-048fc5f441b3',
 '54f45f42-d57f-47ab-a06a-b5aee163c52d',
 '484c6df8-69d0-4df2-88c7-2bf52a2874aa',
 '65728d9f-0887-472f-bb8a-7a54d2e5b288',
 '30d7a098-2d94-41cd-8bd8-d32cfd217e9d',
 '9ddcb9a4-efe9-4b38-9a04-ca3ef79ef6b2',
 '6fc5bfbc-cea3-4752-aa07-596f33e1878b',
 '4fda3aa7-4839-4d55-bc36-68912867cdba',
 '90167fe8-e240-4cea-945c-0ad6b89f81be',
 '589e8f64-47ee-4f7a-84c0-18c273459f5f',
 'bf45d766-460b-44e5-a120-991519ce327a',
 'a34156ca-9a59-413b-840e-0578d406f2be',
 'cab30d0e-4661-431e-b5ba-77c250fd4300',
 '59b36ee8-12c0-454c-8727-4b19cbcdc7aa',
 'f51bf8a2-c0d2-496f-99fe-6b10c4f5e43b',
 '8f401827-ba52-4ddd-afb4-f50dd66a0f7c',
 '9528e2a1-453f-4e97-a6c9-572987b02cb2',
 '67039883-d937-4b49-885f-f67df05107a6',
 '925910a6-6d1e-4576-8f38-7569c0d91226',
 'c47834a8-5ae9-47fe-ad0e-5a7d294a1e1b',
 '25170576-e54d-48a2-a8aa-3a1bfffb9c8c',
 'fa01a78f-83fc-4de9-abd0-6deb1d44a509',
 '18481df9-7deb-4cce-8e59-9605950a6ede',
 'ad783dbd-6c6f-476f-a09b-638b8fb3a682',
 '0cd51389-45f0-

## Total number of documents loaded

In [23]:
vectorstore._collection.count()

46

## Helper function to answer the question by extracting relevant documents from the vectorstore

In [25]:
from langchain.chains import RetrievalQA
def answer_question(question):
  retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
  qa = RetrievalQA.from_chain_type(llm=llm,
                                   chain_type="stuff",
                                   retriever=retriever,
                                   return_source_documents=True)
  result = qa.invoke({"query": question})
  return result['result']

In [26]:
question = "What is Fibromyalgia"
answer_question(question)

'Fibromyalgia is a complex condition characterized by a collection of symptoms that occur together without an obvious or identifiable cause. The main symptom is persistent (long-term) pain throughout many areas of the body, often accompanied by fatigue, poor unrefreshing sleep, and cognitive problems. It is a real condition with underlying changes in the chemistry of the pain system, including changes in chemicals such as Substance P, growth hormone, cortisol, serotonin, and norepinephrine.'

In [27]:
#1. Create .wav file
# Record audio for 5 seconds
audio_data = record(5)

# Save the audio data to a .wav file
with open('recorded_audio.wav', 'wb') as f:
    f.write(audio_data)

print("Audio saved as 'recorded_audio.wav'.")
#2. Convert Speect to text
transcription = transcribe_audio(filename)
#3 Perform RAG
answer_question(transcription)

Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 What is fibromyalgia?


"Fibromyalgia is a syndrome, which means it's a collection of symptoms that occur together without an obvious or identifiable cause. The main symptom is persistent (long-term) pain throughout many areas of the body, often accompanied by fatigue, poor unrefreshing sleep, and cognitive problems."

## Audio enabled RAG Helper Function

In [31]:
def Auio_RAG():
  #1. Create .wav file
  # Record audio for 5 seconds
  print("Start Recording...")
  audio_data = record(5)

  # Save the audio data to a .wav file
  with open('recorded_audio.wav', 'wb') as f:
      f.write(audio_data)

  print("Audio saved as 'recorded_audio.wav'.")
  #2. Convert Speect to text
  transcription = transcribe_audio(filename)
  #3 Perform RAG
  response = answer_question(transcription)
  return response

## Execution 1

In [29]:
Auio_RAG()

Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 What is fibromyalgia?


"Fibromyalgia is a syndrome, which means it's a collection of symptoms that occur together without an obvious or identifiable cause. The main symptom of fibromyalgia is persistent, long-term pain throughout many areas of the body, often accompanied by fatigue, poor unrefreshing sleep, and cognitive problems."

## Execution 2

In [30]:
Auio_RAG()

Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 What are the symptoms associated with fibromyalgia?


"The symptoms associated with fibromyalgia include:\n\n1. Persistent (long-term) pain throughout many areas of the body.\n2. Fatigue\n3. Poor unrefreshing sleep\n4. Cognitive problems\n\nAdditionally, people with fibromyalgia may experience:\n\n1. Altered and 'strange' sensations\n2. Sensitivity to:\n   - Noise\n   - Temperature changes\n   - Smells\n   - Bright lights\n   - Busy places\n\nOther symptoms that may be related to or overlap with fibromyalgia include:\n\n1. Chronic fatigue syndrome (CFS/ME)\n2. Interstitial cystitis\n3. Migraine\n4. TMJ disorders\n5. Dizziness due to vestibular complaints\n6. Neurally mediated hypotension\n7. Burning mouth syndrome\n8. Non-cardiac chest pain\n9. Wheezing\n10. Raynaud's\n11. Hives/welts\n12. Ringing in ears\n13. Vomiting\n14. Heartburn\n15. Oral ulcers\n\nIt's also common for people with fibromyalgia to experience emotional symptoms such as:\n\n1. Low mood\n2. Depression\n3. Frustration\n4. Anxiety\n5. Worry"

## Execution 3

In [32]:
print(Auio_RAG())

Start Recording...
Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 What is cognitive problem?
According to the provided context, cognitive problems refer to difficulties with mental processes, including problems with memory, concentration, finding the words you want, and ordering your thoughts. Some people with fibromyalgia describe this as 'fibro-fog'.


## Execution 4

In [33]:
print(Auio_RAG())

Start Recording...
Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 What are some of the other symptoms that can occur with fibromyalgia?
According to the provided context, some of the other symptoms that can occur with fibromyalgia include:

1. Altered and 'strange' sensations
2. Sensitivity to:
   - Noise
   - Temperature changes
   - Smells
   - Bright lights
   - Busy places
3. Emotional symptoms:
   - Low mood
   - Depression
   - Frustration
   - Anxiety
   - Worry
4. Physical symptoms:
   - Wheezing
   - Raynaud's
   - Hives/welts
   - Ringing in ears
   - Vomiting
   - Heartburn
   - Oral ulcers

Additionally, fibromyalgia can overlap with other conditions such as:

1. Chronic fatigue syndrome (CFS/ME)
2. Interstitial cystitis
3. Migraine
4. TMJ disorders
5. Dizziness due to vestibular complaints
6. Neurally mediated hypotension
7. Burning mouth syndrome
8. Non-cardiac chest pain


## Execution 5

In [35]:
print(Auio_RAG())

Start Recording...
Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 Other conditions that fibromyalgia might have an overlap with.
According to the provided context, fibromyalgia can overlap with the following conditions:

1. Chronic fatigue syndrome (CFS/ME)
2. Interstitial cystitis
3. Migraine
4. TMJ disorders
5. Dizziness due to vestibular complaints
6. Neurally mediated hypotension
7. Burning mouth syndrome
8. Non-cardiac chest pain

Additionally, some symptoms that may occur in people with fibromyalgia, but are not necessarily part of the condition itself, include:

1. Wheezing
2. Raynaud's
3. Hives/welts
4. Ringing in the ears
5. Vomiting
6. Heartburn
7. Oral ulcers


## Execution 6

In [36]:
print(Auio_RAG())

Start Recording...
Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 Possible diagnosis of fibromyalgia.
Based on the provided context, a diagnosis of Fibromyalgia Syndrome (FMS) is made by looking at the overall collection of symptoms that a person has, as there is no single test, scan, or blood test that can confirm the diagnosis. 

Some common symptoms that may lead to a possible diagnosis of fibromyalgia include:

1. Persistent (long-term) pain throughout many areas of the body.
2. Fatigue
3. Poor unrefreshing sleep
4. Cognitive problems

A doctor, typically a GP, will also need to rule out other treatable causes or conditions that might account for the pain and other symptoms.


## Execution 7

In [37]:
print(Auio_RAG())

Start Recording...
Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 How fibromyalgia can affect people?
According to the provided context, Fibromyalgia Syndrome (FMS) can affect people in various ways. Some common effects include:

1. Difficulty doing things that matter: FMS can make it challenging for people to perform daily activities and maintain their usual lifestyle.
2. Emotional impact: People with FMS may experience low mood, depression, frustration, anxiety, and worry.
3. Different experiences: Everyone is affected differently by FMS, and some people may maintain a fairly full and active life, while others may find it more challenging.
4. Impact on daily life: FMS symptoms can be influenced by factors such as stress levels, physical activity, weather changes, and other illnesses or pain.
5. Hormonal changes: In women, hormonal changes during menstruation and menopause may affect FMS symptoms.

Additionally, people with FMS may experience a range of symptoms, including:

* Persistent pain thr

## Execution 8

In [38]:
print(Auio_RAG())

Start Recording...
Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 How can we prevent fibromyalgia?
Unfortunately, I don't know the answer to that question. The provided context does not mention any specific ways to prevent fibromyalgia, and it states that "it is not fully understood what causes Fibromyalgia Syndrome."


## Execution 9

In [39]:
print(Auio_RAG())

Start Recording...
Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 What causes fibromyalgia?
It is not fully understood what causes Fibromyalgia Syndrome. It is most likely that it is caused by a number of different factors that come together to trigger the onset of the condition.


## Execution 10

In [40]:
print(Auio_RAG())

Start Recording...
Recording...


<IPython.core.display.Javascript object>

Done recording!
Audio saved as 'recorded_audio.wav'.
 What happens in fibromyalgia syndrome?
In Fibromyalgia Syndrome (FMS), there are changes in the nervous system that affect the way nerve messages are processed. It's not a structural change that can be seen on scans, but rather a change in the way the system functions, similar to a change in software. This leads to a variety of symptoms, including:

* Persistent pain throughout many areas of the body
* Fatigue
* Poor unrefreshing sleep
* Cognitive problems
* Altered and 'strange' sensations
* Sensitivity to noise, temperature changes, smells, bright lights, and busy places

These changes in the nervous system cause the body to be in a state of heightened sensitivity, leading to the characteristic widespread pain and other symptoms of FMS.
